In [1]:
import tflearn as tf
import tensorflow as tff
import pandas as pd
import numpy as np
import seaborn as sns

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print(test_data.shape, train_data.shape)
test_data['SalePrice'] = 0
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 999
all_data = pd.concat([train_data, test_data], axis=0)

(1459, 80) (1460, 81)


In [3]:
def clean_data(data):
        data["LotFrontage"].fillna(data["LotFrontage"].mean(), inplace=True)
        data["MasVnrArea"].fillna(data["MasVnrArea"].mean(), inplace=True)
        data["GarageYrBlt"].fillna(data["GarageYrBlt"].mean(), inplace=True)
        data = pd.get_dummies(data, columns=['MSZoning', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'], drop_first=True)
        print(data.shape)
        return data

all_data = clean_data(all_data)
train_data = all_data.iloc[:1459]
test_data = all_data.iloc[1459:]
ids = all_data.Id.iloc[1460:]
test_data = test_data.drop(test_data.columns[0], axis=1)
train_data = train_data.drop(train_data.columns[0], axis=1)
train_labels = pd.DataFrame()
train_labels = train_data['SalePrice'].copy()
train_data = train_data.drop(train_data.columns[-1], axis=1)
test_data = train_data.drop(train_data.columns[-1], axis=1)
train_data.shape
test_data.shape

(2919, 247)


(1459, 244)

In [4]:
train_labels = np.reshape(train_labels, (-1, 1))

/Users/ewallner/Library/Python/3.6/lib/python/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


In [5]:
train_labels.shape

(1459, 1)

In [6]:
train_data = np.array(train_data, dtype=np.int32)
test_data = np.array(train_data, dtype=np.int32)
train_labels = np.array(train_labels, dtype=np.int32)

In [ ]:
train_labels

array([[208500],
       [181500],
       [223500],
       ..., 
       [210000],
       [266500],
       [142125]], dtype=int32)

tff.reset_default_graph()
#r2 = tf.R2()
#net = tf.input_data(shape=[None, train_data.shape[1]])
#net = tf.fully_connected(net, 30)
#net = tf.fully_connected(net, 10)
#net = tf.fully_connected(net, 1, activation='linear')
#sgd = tf.SGD()
#net = tf.regression(net)
#model = tf.DNN(net)
r2 = tf.R2()
net = tf.input_data(shape=[None, train_data.shape[1]])
net = tf.fully_connected(net, 30, activation='linear')
net = tf.fully_connected(net, 10, activation = 'linear')
net = tf.fully_connected(net, 1, activation = 'linear')
sgd = tf.SGD(learning_rate=0.1, lr_decay = 0.01, decay_step=100)
net = tf.regression(net, optimizer=sgd, loss='mean_square', metric=r2)
model = tf.DNN(net)
model.fit(train_data, train_labels, show_metric=True, validation_set=0.2, shuffle = True, n_epoch=50)

tff.reset_default_graph()
# Network building
net = tf.input_data([None, train_data.shape[1]])
net = tf.fully_connected(net, 64, activation='linear',
                                 regularizer='L2', weight_decay=0.0005)
#net = tf.embedding(net, input_dim=100, output_dim=128)
#net = tf.lstm(net, 128, dropout=0.1)
net = tf.fully_connected(net, 1, activation='linear')
net = tf.regression(net, optimizer=tf.optimizers.AdaGrad(learning_rate=0.01, initial_accumulator_value=0.01), loss='mean_square', learning_rate=0.05)

# Training
model = tf.DNN(net, tensorboard_verbose=0)
model.fit(train_data, train_labels, show_metric=True, n_epoch=2000, batch_size=100)

preds_DNN = model.predict(test_data)
preds_DNN = np.exp(preds_DNN)
preds_DNN = preds_DNN.reshape(-1,)

print(ids)
print(preds_DNN)


output = pd.DataFrame({"id":ids, "SalePrice": preds_DNN})
output.to_csv('output2.csv', index=False)
preds_DNN

In [ ]:
tff.reset_default_graph()
r2 = tf.R2()
net = tf.input_data(shape=[None, train_data.shape[1]])
net = tf.fully_connected(net, 30, activation='linear')
net = tf.fully_connected(net, 10, activation = 'linear')
net = tf.fully_connected(net, 1, activation = 'linear')
sgd = tf.SGD(learning_rate=0.1, lr_decay = 0.01, decay_step=100)
net = tf.regression(net, optimizer=sgd, loss='mean_square', metric=r2)
model = tf.DNN(net)

model.fit(train_data, train_labels, show_metric=True, validation_set=0.2, shuffle = True, batch_size=1460, n_epoch=2000)

---------------------------------
Run id: TC3G76
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name StandardError/ (raw) is illegal; using StandardError/__raw_ instead.
---------------------------------
Training samples: 1167
Validation samples: 292
--
Training Step: 1  | time: 1.068s
| SGD | epoch: 001 | loss: 0.00000 - R2: 0.0000 | val_loss: 39477420032.00000 - val_acc: 0.0000 -- iter: 1167/1167
--
Training Step: 2  | total loss: 34938777600.00000 | time: 1.007s
| SGD | epoch: 002 | loss: 34938777600.00000 - R2: 0.0000 | val_loss: 32734269440.00000 - val_acc: 0.0090 -- iter: 1167/1167
--
Training Step: 3  | total loss: 37729742848.00000 | time: 1.009s
| SGD | epoch: 003 | loss: 37729742848.00000 - R2: 0.0000 | val_loss: 15640097792.00000 - val_acc: 0.1402 -- iter: 1167/1167
--
Training Step: 4  | total loss: 34157819904.00000 | time: 1.008s
| SGD | epoch: 004 | loss: 34157819904.00000 - R2: 0.0068 | val_loss: 258702016.00000 - val_acc: 0.8456 -- iter: 1167/1167
--
Trainin

Training Step: 40  | total loss: 1233602944.00000 | time: 1.007s
| SGD | epoch: 040 | loss: 1233602944.00000 - R2: 0.9501 | val_loss: 44130392.00000 - val_acc: 1.0665 -- iter: 1167/1167
--
Training Step: 41  | total loss: 1088857344.00000 | time: 1.007s
| SGD | epoch: 041 | loss: 1088857344.00000 - R2: 0.9222 | val_loss: 411736224.00000 - val_acc: 0.8076 -- iter: 1167/1167
--
Training Step: 42  | total loss: 1130955776.00000 | time: 1.008s
| SGD | epoch: 042 | loss: 1130955776.00000 - R2: 0.9494 | val_loss: 29233030.00000 - val_acc: 1.0535 -- iter: 1167/1167
--
Training Step: 43  | total loss: 1001451392.00000 | time: 1.007s
| SGD | epoch: 043 | loss: 1001451392.00000 - R2: 0.9246 | val_loss: 368313344.00000 - val_acc: 0.8175 -- iter: 1167/1167
--
Training Step: 44  | total loss: 1303673216.00000 | time: 1.011s
| SGD | epoch: 044 | loss: 1303673216.00000 - R2: 0.9473 | val_loss: 18452940.00000 - val_acc: 1.0418 -- iter: 1167/1167
--
Training Step: 45  | total loss: 1142610816.00000 | t

Training Step: 80  | total loss: 1094381312.00000 | time: 1.007s
| SGD | epoch: 080 | loss: 1094381312.00000 - R2: 1.0041 | val_loss: 9800710.00000 - val_acc: 1.0298 -- iter: 1167/1167
--
Training Step: 81  | total loss: 984221888.00000 | time: 1.008s
| SGD | epoch: 081 | loss: 984221888.00000 - R2: 1.0022 | val_loss: 2989498.75000 - val_acc: 0.9860 -- iter: 1167/1167
--
Training Step: 82  | total loss: 944039552.00000 | time: 1.008s
| SGD | epoch: 082 | loss: 944039552.00000 - R2: 1.0032 | val_loss: 8644911.00000 - val_acc: 1.0278 -- iter: 1167/1167
--
Training Step: 83  | total loss: 850082368.00000 | time: 1.010s
| SGD | epoch: 083 | loss: 850082368.00000 - R2: 1.0016 | val_loss: 2504938.00000 - val_acc: 0.9878 -- iter: 1167/1167
--
Training Step: 84  | total loss: 839028480.00000 | time: 1.009s
| SGD | epoch: 084 | loss: 839028480.00000 - R2: 1.0052 | val_loss: 7654705.00000 - val_acc: 1.0259 -- iter: 1167/1167
--
Training Step: 85  | total loss: 755528832.00000 | time: 1.007s
| SG

Training Step: 121  | total loss: 939607296.00000 | time: 1.009s
| SGD | epoch: 121 | loss: 939607296.00000 - R2: 1.0006 | val_loss: 909380.81250 - val_acc: 0.9973 -- iter: 1167/1167
--
Training Step: 122  | total loss: 943162624.00000 | time: 1.011s
| SGD | epoch: 122 | loss: 943162624.00000 - R2: 1.0025 | val_loss: 957433.31250 - val_acc: 1.0036 -- iter: 1167/1167
--
Training Step: 123  | total loss: 849078848.00000 | time: 1.007s
| SGD | epoch: 123 | loss: 849078848.00000 - R2: 1.0021 | val_loss: 903176.25000 - val_acc: 0.9973 -- iter: 1167/1167
--
Training Step: 124  | total loss: 1046482624.00000 | time: 1.010s
| SGD | epoch: 124 | loss: 1046482624.00000 - R2: 1.0019 | val_loss: 916352.68750 - val_acc: 1.0030 -- iter: 1167/1167
--
Training Step: 125  | total loss: 942065088.00000 | time: 1.011s
| SGD | epoch: 125 | loss: 942065088.00000 - R2: 1.0015 | val_loss: 895693.37500 - val_acc: 0.9973 -- iter: 1167/1167
--
Training Step: 126  | total loss: 943467520.00000 | time: 1.007s
| S

Training Step: 162  | total loss: 2218525440.00000 | time: 1.007s
| SGD | epoch: 162 | loss: 2218525440.00000 - R2: 0.9968 | val_loss: 787618.62500 - val_acc: 0.9987 -- iter: 1167/1167
--
Training Step: 163  | total loss: 1996884608.00000 | time: 1.007s
| SGD | epoch: 163 | loss: 1996884608.00000 - R2: 0.9970 | val_loss: 823575.93750 - val_acc: 0.9977 -- iter: 1167/1167
--
Training Step: 164  | total loss: 2099261184.00000 | time: 1.008s
| SGD | epoch: 164 | loss: 2099261184.00000 - R2: 0.9974 | val_loss: 789575.43750 - val_acc: 0.9986 -- iter: 1167/1167
--
Training Step: 165  | total loss: 1889546496.00000 | time: 1.011s
| SGD | epoch: 165 | loss: 1889546496.00000 - R2: 0.9976 | val_loss: 823481.31250 - val_acc: 0.9977 -- iter: 1167/1167
--
Training Step: 166  | total loss: 2034677376.00000 | time: 1.007s
| SGD | epoch: 166 | loss: 2034677376.00000 - R2: 0.9978 | val_loss: 791481.87500 - val_acc: 0.9985 -- iter: 1167/1167
--
Training Step: 167  | total loss: 1831420800.00000 | time: 1

...

Training Step: 1622  | total loss: 889798144.00000 | time: 1.007s
| SGD | epoch: 1622 | loss: 889798144.00000 - R2: 1.0002 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1623  | total loss: 801024960.00000 | time: 1.007s
| SGD | epoch: 1623 | loss: 801024960.00000 - R2: 1.0001 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1624  | total loss: 852793792.00000 | time: 1.009s
| SGD | epoch: 1624 | loss: 852793792.00000 - R2: 0.9972 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1625  | total loss: 767721024.00000 | time: 1.007s
| SGD | epoch: 1625 | loss: 767721024.00000 - R2: 0.9974 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1626  | total loss: 716807296.00000 | time: 1.010s
| SGD | epoch: 1626 | loss: 716807296.00000 - R2: 0.9973 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1627  | total loss: 645333184.00000 | time: 1

Training Step: 1662  | total loss: 885924800.00000 | time: 1.010s
| SGD | epoch: 1662 | loss: 885924800.00000 - R2: 0.9979 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1663  | total loss: 797538944.00000 | time: 1.009s
| SGD | epoch: 1663 | loss: 797538944.00000 - R2: 0.9980 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1664  | total loss: 806302720.00000 | time: 1.007s
| SGD | epoch: 1664 | loss: 806302720.00000 - R2: 0.9989 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1665  | total loss: 725879040.00000 | time: 1.007s
| SGD | epoch: 1665 | loss: 725879040.00000 - R2: 0.9989 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1666  | total loss: 743329088.00000 | time: 1.007s
| SGD | epoch: 1666 | loss: 743329088.00000 - R2: 0.9986 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1667  | total loss: 669202816.00000 | time: 1

Training Step: 1702  | total loss: 1001582848.00000 | time: 1.008s
| SGD | epoch: 1702 | loss: 1001582848.00000 - R2: 0.9977 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1703  | total loss: 901631168.00000 | time: 1.008s
| SGD | epoch: 1703 | loss: 901631168.00000 - R2: 0.9978 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1704  | total loss: 1134236160.00000 | time: 1.007s
| SGD | epoch: 1704 | loss: 1134236160.00000 - R2: 1.0003 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1705  | total loss: 1021019136.00000 | time: 1.007s
| SGD | epoch: 1705 | loss: 1021019136.00000 - R2: 1.0002 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1706  | total loss: 996135680.00000 | time: 1.008s
| SGD | epoch: 1706 | loss: 996135680.00000 - R2: 1.0014 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1707  | total loss: 896728704.00000 | t

Training Step: 1742  | total loss: 1939022592.00000 | time: 1.007s
| SGD | epoch: 1742 | loss: 1939022592.00000 - R2: 1.0027 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1743  | total loss: 1745326848.00000 | time: 1.009s
| SGD | epoch: 1743 | loss: 1745326848.00000 - R2: 1.0023 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1744  | total loss: 1843372032.00000 | time: 1.011s
| SGD | epoch: 1744 | loss: 1843372032.00000 - R2: 1.0008 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1745  | total loss: 1659241472.00000 | time: 1.007s
| SGD | epoch: 1745 | loss: 1659241472.00000 - R2: 1.0006 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1746  | total loss: 1576873600.00000 | time: 1.009s
| SGD | epoch: 1746 | loss: 1576873600.00000 - R2: 1.0002 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1747  | total loss: 1419392768.0000

Training Step: 1782  | total loss: 626922752.00000 | time: 1.010s
| SGD | epoch: 1782 | loss: 626922752.00000 - R2: 0.9989 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1783  | total loss: 564437120.00000 | time: 1.008s
| SGD | epoch: 1783 | loss: 564437120.00000 - R2: 0.9989 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1784  | total loss: 796841216.00000 | time: 1.007s
| SGD | epoch: 1784 | loss: 796841216.00000 - R2: 1.0030 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1785  | total loss: 717363712.00000 | time: 1.010s
| SGD | epoch: 1785 | loss: 717363712.00000 - R2: 1.0026 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1786  | total loss: 763403456.00000 | time: 1.007s
| SGD | epoch: 1786 | loss: 763403456.00000 - R2: 1.0000 | val_loss: 798799.81250 - val_acc: 0.9979 -- iter: 1167/1167
--
Training Step: 1787  | total loss: 687269696.00000 | time: 1

In [ ]:
preds_DNN = model.predict(test_data)
preds_DNN = preds_DNN.reshape(-1,)
output = pd.DataFrame({"id":ids})
output["SalePrice"] = preds_DNN
#output_id = pd.DataFrame({"id":ids})
output.to_csv('output2.csv', index=False)
output